<a href="https://colab.research.google.com/github/VIVEKKUMARSINGH72/GenerativeAI/blob/main/Amazon_Return_Policy_Assitant1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio openai faiss-gpu tiktoken sentence-transformers



ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving amazon_return_policy.txt to amazon_return_policy.txt


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.amazon.in/gp/help/customer/display.html?nodeId=202111910"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

policy_text = "\n".join([p.get_text(strip=True) for p in soup.find_all("p")])
with open("amazon_return_policy.txt", "w", encoding="utf-8") as f:
    f.write(policy_text)

In [ ]:
sample_text = """
Amazon allows returns within 30 days of receipt of shipment.
Items must be in new and unused condition.
Some products (like software, groceries, etc.) may not be eligible.
Refunds are processed within 3–5 business days after the returned item is received.
Shipping costs may be deducted unless the return is Amazon’s fault.
"""

with open("amazon_return_policy.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)
print("✅ Sample amazon_return_policy.txt created.")

✅ Sample amazon_return_policy.txt created.


In [ ]:
import os

# Check if file exists and has content
if not os.path.exists("amazon_return_policy.txt") or os.path.getsize("amazon_return_policy.txt") == 0:
    print("⚠️ File is missing or empty. Creating sample policy file...")
    sample_text = """
    Amazon allows returns within 30 days of receipt.
    Some categories like groceries and software may not be eligible.
    Refunds take 3–5 business days after the item is received.
    Return shipping costs may apply unless the error was Amazon's.
    """
    with open("amazon_return_policy.txt", "w", encoding="utf-8") as f:
        f.write(sample_text)
else:
    print("✅ File found and not empty.")

✅ File found and not empty.


In [ ]:
with open("amazon_return_policy.txt", "r", encoding="utf-8") as f:
    text_chunks = [line.strip() for line in f if line.strip()]

print("📝 Loaded", len(text_chunks), "chunks")

if not text_chunks:
    raise ValueError("❌ Text file is still empty after reading. Please check content.")


📝 Loaded 5 chunks


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 64.7 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(text_chunks, convert_to_tensor=False)

if len(embeddings) == 0 or len(embeddings[0]) == 0:
    raise ValueError("❌ Embedding failed. Check input text.")

embeddings = np.array(embeddings).astype("float32")
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

faiss.write_index(index, "policy.index")
print("✅ FAISS index created and saved as 'policy.index'")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS index created and saved as 'policy.index'


In [ ]:
import gradio as gr
import faiss
from sentence_transformers import SentenceTransformer
import os

text_file = "amazon_return_policy.txt"
index_file = "policy.index"

with open(text_file, "r", encoding="utf-8") as f:
    text_chunks = [line.strip() for line in f if line.strip()]

embed_model = SentenceTransformer('all-MiniLM-L6-v2')
index = faiss.read_index(index_file)

def retrieve_context(query, k=3):
    q_emb = embed_model.encode([query])
    distances, indices = index.search(q_emb, k)
    return "\n".join([text_chunks[i] for i in indices[0]])

def respond(message, history):
    context = retrieve_context(message)
    response = f"📄 Based on Amazon's policy:\n{context}"
    return response

chat_interface = gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(type="messages", label="Amazon Return Policy Assistant"),
    title="📦 Amazon Return Policy Assistant",
    description="Ask any question about Amazon's return policy."
)

chat_interface.launch(share=True, debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:322: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cbef77857c90167e0e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
